### Importing the Libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
tf.__version__ 

'2.12.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
training_set = train_datagen.flow_from_directory('./drive/MyDrive/dataSet/trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 7106 images belonging to 27 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('./drive/MyDrive/dataSet/testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 3560 images belonging to 27 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [ ]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [ ]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [ ]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [ ]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [ ]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [ ]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [ ]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [ ]:
classifier.fit(training_set,
                  epochs = 10,
                  validation_data = test_set)

Epoch 1/10
711/711 [==============================] - 1752s 2s/step - loss: 2.8609 - accuracy: 0.1435 - val_loss: 1.9538 - val_accuracy: 0.3778
Epoch 2/10
711/711 [==============================] - 225s 316ms/step - loss: 1.8961 - accuracy: 0.3714 - val_loss: 1.3120 - val_accuracy: 0.5680
Epoch 3/10
711/711 [==============================] - 236s 333ms/step - loss: 1.5663 - accuracy: 0.4733 - val_loss: 1.1294 - val_accuracy: 0.6211
Epoch 4/10
711/711 [==============================] - 228s 320ms/step - loss: 1.3708 - accuracy: 0.5360 - val_loss: 1.0159 - val_accuracy: 0.6685
Epoch 5/10
711/711 [==============================] - 242s 341ms/step - loss: 1.2197 - accuracy: 0.5812 - val_loss: 0.9510 - val_accuracy: 0.6876
Epoch 6/10
711/711 [==============================] - 242s 340ms/step - loss: 1.1126 - accuracy: 0.6257 - val_loss: 0.9361 - val_accuracy: 0.6868
Epoch 7/10
711/711 [==============================] - 243s 342ms/step - loss: 1.0273 - accuracy: 0.6454 - val_loss: 0.9748 - v

#### Saving the Model

In [ ]:
model_json = classifier.to_json()
with open("model_new_3.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new_3.h5')
print('Weights saved')

Model Saved
Weights saved
